# Code to detect the speed and the distance traveled by the puck throughout the video.

In [2]:
from roboflow import Roboflow
import cv2
import numpy as np

# Replace "your_api_key" with your Roboflow API key
# Initialize Roboflow client
rf = Roboflow(api_key="XLIonB4LFouarhOUrUn5")
project = rf.workspace().project("icehockey")
model = project.version(1).model

# Open a video capture
cap = cv2.VideoCapture(r"C:\Users\mahes\Downloads\Hockey Drills.mp4")

# Set the desired frame rate (30 frames per second)
cap.set(cv2.CAP_PROP_FPS, 30)

# Initialize variables for tracking speed and distance
total_distance = 0
start_frame = None
prev_puck_center = None  # Initialize prev_puck_center

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Make predictions using Roboflow model
    result = model.predict(frame, confidence=50, overlap=0).json()

    # Visualize prediction on the frame
    for prediction in result["predictions"]:
        confidence = prediction.get("confidence", 0.0)
        x, y, w, h = prediction.get("xmin", 0), prediction.get("ymin", 0), prediction.get("width", 0), prediction.get("height", 0)

        # Calculate puck center
        puck_center = np.array([(x + x + w) / 2, (y + y + h) / 2])

        # Update distance travelled
        if start_frame is not None and prev_puck_center is not None:
            total_distance += np.linalg.norm(puck_center - prev_puck_center)

        # Save current puck center for the next iteration
        prev_puck_center = puck_center

    # Check if this is the first frame to set the start frame
    if start_frame is None:
        start_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

# Calculate the average speed
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
average_speed = total_distance / total_frames

# Print the results
print(f"Average Speed of the Puck: {average_speed} pixels per frame")
print(f"Overall Distance Travelled by the Puck: {total_distance} pixels")

# Release the video capture object
cap.release()


loading Roboflow workspace...
loading Roboflow project...
Average Speed of the Puck: 0.15886426668315748 pixels per frame
Overall Distance Travelled by the Puck: 144.248754148307 pixels
